# LENDING CLUB CASE

In [ ]:
# Importing essential libs and modules for analysis
import pandas as pd #To work with dataset
import numpy as np #Mathematical operation rich lib
import seaborn as sns #Graphical library 
import matplotlib.pyplot as plt #Used as backbone for seaborn

In [ ]:
# Reading dataset
df = pd.read_csv('loan.csv')

In [ ]:
# Checking data for content
df.head(10)

In [ ]:
#Reading insights
df.describe()

In [ ]:
#Checking for null values
df.isnull().sum()

In [ ]:
#Deleting columns with null values having more than 85% of null values
for col in df.columns:
    if df[col].isnull().sum() >= (len(df.index)*85)/100 :
        del df[col]
    

In [ ]:
#Checking updated null values
df.isnull().sum()

In [ ]:
#Check dataset info for dtypes and col number for reference 
df.info()

In [ ]:
#Using dtale for better insights as it's hoghly potent with great functionalities
import dtale
d = dtale.show(df)
d